In [1]:
import os
import pandas as pd

# 資料夾路徑
folder_path = r"D:\works\Data_analysis\projects\balance_beam\2023May\preprocessed_DLC_data\pose_estimation"

# 儲存結果的DataFrame
result_df = pd.DataFrame(columns=['檔名', '總歷時', '在beam上的歷時'])

# 處理資料夾中的每個檔案
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        
        # 讀取資料並指定多層次標題
        df = pd.read_csv(file_path, header=[0, 1, 2])
        
        column_start_x = df[('DLC_resnet50_Balance_BeamMay29shuffle1_150000', 'start', 'x')]
        column_lateral_body_x = df[('DLC_resnet50_Balance_BeamMay29shuffle1_150000', 'lateral_body', 'x')]
        
        # 使用布林索引比較兩個欄位的值
        filtered_rows = df[column_lateral_body_x >= column_start_x]
        
        # 計算總歷時和在beam上的歷時
        total_duration = len(column_lateral_body_x)
        on_beam_duration = len(filtered_rows)
        
        # 建立包含結果的DataFrame
        result_row = pd.DataFrame({
            '檔名': [file_name],
            '總歷時': [total_duration],
            '在beam上的歷時': [on_beam_duration]
        })
        
        # 將結果加入結果DataFrame
        result_df = pd.concat([result_df, result_row])


# 讀取結果檔案
df = result_df

# 提取日期和treatment的資訊
df['日期'] = df['檔名'].str.extract(r'(\d{4})')
df['treatment'] = df['檔名'].str.extract(r'_([A-Za-z]+)_')

# 建立Excel寫入器
writer = pd.ExcelWriter('duration_result.xlsx')


 
# 分組寫入不同分頁
for (date, treatment), group in df.groupby(['日期', 'treatment']):
    sheet_name = f"{date}_{treatment}"
    group.to_excel(writer, sheet_name=sheet_name, index=False)

# 保存Excel文件
writer.save()


d:\App\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
